In [1]:
import pandas as pd
from ydata_profiling import ProfileReport
from ydata_profiling.utils.cache import cache_file

import phik
from phik.report import plot_correlation_matrix
from phik import report

import matplotlib.pyplot as plt

%matplotlib inline

#print all output in a cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"



In [2]:
# Read in the satellite drag database
fn = 'D:\\data\\VL_sdrag\\satdrag_database_zlib.hdf5'
columns = ['400kmDensity','SYM/H_INDEX_nT', '1-M_AE_nT', 'DAILY_SUNSPOT_NO_', 'DAILY_F10.7_', 
       'SOLAR_LYMAN-ALPHA_W/m^2', 'mg_index (core to wing ratio (unitless))',
       'irradiance (W/m^2/nm)', 'storm', 'storm phase']
df = pd.read_hdf(fn,columns=columns)

df['d_diff'] = df['400kmDensity'].diff()
df = df.dropna()


In [3]:
df.columns

Index(['400kmDensity', 'SYM/H_INDEX_nT', '1-M_AE_nT', 'DAILY_SUNSPOT_NO_',
       'DAILY_F10.7_', 'SOLAR_LYMAN-ALPHA_W/m^2',
       'mg_index (core to wing ratio (unitless))', 'irradiance (W/m^2/nm)',
       'storm', 'storm phase', 'd_diff'],
      dtype='object')

In [4]:
df.head()

,400kmDensity,SYM/H_INDEX_nT,1-M_AE_nT,DAILY_SUNSPOT_NO_,DAILY_F10.7_,SOLAR_LYMAN-ALPHA_W/m^2,mg_index (core to wing ratio (unitless)),irradiance (W/m^2/nm),storm,storm phase,d_diff
1,1.276178e-12,-2.0,92.0,70.0,94.5,0.00697,0.26771,0.005673,1,1,-2.124240e-13
2,1.230694e-12,-2.0,130.0,70.0,94.5,0.00697,0.26771,0.005673,1,1,-4.548400e-14
3,1.220807e-12,-3.0,85.0,70.0,94.5,0.00697,0.26771,0.005673,1,1,-9.887000e-15
4,1.304111e-12,-3.0,75.0,70.0,94.5,0.00697,0.26771,0.005673,1,1,8.330400e-14
5,1.465795e-12,-3.0,98.0,70.0,94.5,0.00697,0.26771,0.005673,1,1,1.616840e-13


In [5]:
df.describe()

,400kmDensity,SYM/H_INDEX_nT,1-M_AE_nT,DAILY_SUNSPOT_NO_,DAILY_F10.7_,SOLAR_LYMAN-ALPHA_W/m^2,mg_index (core to wing ratio (unitless)),irradiance (W/m^2/nm),storm,storm phase,d_diff
count,4.670940e+06,4.670940e+06,4.670940e+06,4.670940e+06,4.670940e+06,4.670940e+06,4.670940e+06,4.670940e+06,4.670940e+06,4.670940e+06,4.670940e+06
mean,1.484615e-12,-1.141293e+01,1.763493e+02,4.752017e+01,9.761624e+01,6.825335e-03,2.685655e-01,5.515092e-03,1.216672e-01,4.610534e-01,-1.020210e-19
std,1.470658e-12,1.914744e+01,2.141171e+02,5.038690e+01,5.429083e+01,7.783128e-04,4.569255e-03,5.460839e-04,9.925711e-01,1.343722e+00,1.700765e-13
min,7.457587e-20,-4.900000e+02,1.000000e+00,0.000000e+00,6.510000e+01,5.880000e-03,2.629600e-01,4.873058e-03,-1.000000e+00,-1.000000e+00,-1.175625e-11
25%,4.975017e-13,-1.800000e+01,3.900000e+01,9.000000e+00,7.150000e+01,6.174000e-03,2.647066e-01,5.050188e-03,-1.000000e+00,-1.000000e+00,-3.611200e-14
50%,9.641593e-13,-8.000000e+00,8.800000e+01,3.100000e+01,8.520000e+01,6.577000e-03,2.670400e-01,5.328221e-03,1.000000e+00,1.000000e+00,4.781500e-16
75%,1.931388e-12,-1.000000e+00,2.360000e+02,7.300000e+01,1.111000e+02,7.319000e-03,2.714976e-01,5.854947e-03,1.000000e+00,2.000000e+00,3.793913e-14
max,2.631810e-11,1.510000e+02,4.192000e+03,2.810000e+02,9.999000e+02,9.751000e-03,2.849400e-01,7.349350e-03,1.000000e+00,2.000000e+00,1.223247e-11


In [6]:
df.shape

(4670940, 11)

In [7]:
correlations = {"auto": {"calculate": True},
        "pearson": {"calculate": True},
        "spearman": {"calculate": True},
        "kendall": {"calculate": True},
        "phi_k": {"calculate": True},
        "cramers": {"calculate": True},
    }
profile = ProfileReport(df.sample(1000000), correlations=correlations)
profile.to_file("all_data.html")

cc = profile.description_set["correlations"]
cc["phi_k"]["400kmDensity"]

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

c:\Users\krmurph1\Anaconda3\envs\satdrag\lib\site-packages\ydata_profiling\model\pandas\discretize_pandas.py:52: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  discretized_df.loc[:, column] = self._discretize_column(
c:\Users\krmurph1\Anaconda3\envs\satdrag\lib\site-packages\scipy\stats\_stats_py.py:5218: RuntimeWarning: overflow encountered in longlong_scalars
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

400kmDensity                                1.000000
SYM/H_INDEX_nT                              0.599569
1-M_AE_nT                                   0.286666
DAILY_SUNSPOT_NO_                           0.614090
DAILY_F10.7_                                0.507440
SOLAR_LYMAN-ALPHA_W/m^2                     0.680973
mg_index (core to wing ratio (unitless))    0.663269
irradiance (W/m^2/nm)                       0.685398
storm                                       0.196851
storm phase                                 0.177687
d_diff                                      0.325193
Name: 400kmDensity, dtype: float64

# Look at the profiles for storm break down

In [15]:
# storm/quiet data 
st_t = (df['storm']>0.95) & (df['storm']<1.05) # storm times
qt_t = (df['storm']<0) # quiet times
mn_p = (df['storm phase']>0.95) & (df['storm phase']<1.05) # main phase
rc_p = (df['storm phase']>1.95) & (df['storm phase']<2.05) # recovery phase

In [16]:
# sanity check that the data is the correct size
print(df[st_t].shape)
print(df[qt_t].shape)
print(df[mn_p].shape)
print(df[rc_p].shape)
print(df.shape)

df[st_t].shape[0]+df[qt_t].shape[0]
df[qt_t].shape[0]+df[mn_p].shape[0]+df[rc_p].shape[0]

(2619620, 11)
(2051320, 11)
(1034367, 11)
(1585253, 11)
(4670940, 11)


4670940

4670940

In [17]:
st_ii = [st_t,qt_t,mn_p,rc_p] # list of indexes to loop over
st_tt = ['storm','quiet','main','recovery'] # list of names

for sind, stype in zip(st_ii,st_tt):
    
    profile = ProfileReport(df[sind].sample(1000000), correlations=correlations)
    profile.to_file(f'{stype}_output.html')

    print(f'{stype}_output.html')
    cc = profile.description_set["correlations"]
    cc["phi_k"]["400kmDensity"]

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

c:\Users\krmurph1\Anaconda3\envs\satdrag\lib\site-packages\ydata_profiling\model\pandas\discretize_pandas.py:52: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  discretized_df.loc[:, column] = self._discretize_column(
c:\Users\krmurph1\Anaconda3\envs\satdrag\lib\site-packages\scipy\stats\_stats_py.py:5218: RuntimeWarning: overflow encountered in longlong_scalars
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

...

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

recovery_output.html


400kmDensity                                1.000000
SYM/H_INDEX_nT                              0.617370
1-M_AE_nT                                   0.194703
DAILY_SUNSPOT_NO_                           0.591789
DAILY_F10.7_                                0.478831
SOLAR_LYMAN-ALPHA_W/m^2                     0.657249
mg_index (core to wing ratio (unitless))    0.644341
irradiance (W/m^2/nm)                       0.665928
d_diff                                      0.230440
Name: 400kmDensity, dtype: float64

# What does the 'all' look like when we deal with class imbalance?

In [20]:
#reg_df =df[st_t].sample(500000)
reg_df = pd.concat([df[st_t].sample(500000),df[qt_t].sample(500000)])
reg_df.describe()

,400kmDensity,SYM/H_INDEX_nT,1-M_AE_nT,DAILY_SUNSPOT_NO_,DAILY_F10.7_,SOLAR_LYMAN-ALPHA_W/m^2,mg_index (core to wing ratio (unitless)),irradiance (W/m^2/nm),storm,storm phase,d_diff
count,1.000000e+06,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1.000000e+06
mean,1.448488e-12,-10.740491,169.020393,46.338448,96.671341,0.006805,0.268475,0.005500,0.000000,0.302266,1.675198e-16
std,1.453106e-12,18.423993,208.216124,49.952263,52.488133,0.000773,0.004557,0.000543,1.000001,1.347381,1.628727e-13
min,4.683731e-16,-487.000000,1.000000,0.000000,65.100000,0.005880,0.262960,0.004873,-1.000000,-1.000000,-7.241425e-12
25%,4.805903e-13,-17.000000,37.000000,5.000000,71.200000,0.006154,0.264674,0.005036,-1.000000,-1.000000,-3.532432e-14
50%,9.313840e-13,-8.000000,82.000000,30.000000,84.200000,0.006554,0.266948,0.005316,0.000000,0.000000,4.572000e-16
75%,1.872047e-12,-1.000000,223.000000,71.000000,109.900000,0.007295,0.271344,0.005841,1.000000,2.000000,3.702820e-14
max,2.430800e-11,143.000000,4174.000000,281.000000,999.900000,0.009751,0.284940,0.007349,1.000000,2.000000,8.884224e-12


In [21]:

profile = ProfileReport(reg_df, correlations=correlations)
profile.to_file('qt_st_output.html')

print('qt_st_output.html')
cc = profile.description_set["correlations"]
cc["phi_k"]["400kmDensity"]

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

c:\Users\krmurph1\Anaconda3\envs\satdrag\lib\site-packages\ydata_profiling\model\pandas\discretize_pandas.py:52: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  discretized_df.loc[:, column] = self._discretize_column(
c:\Users\krmurph1\Anaconda3\envs\satdrag\lib\site-packages\scipy\stats\_stats_py.py:5218: RuntimeWarning: overflow encountered in longlong_scalars
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

qt_st_output.html


400kmDensity                                1.000000
SYM/H_INDEX_nT                              0.569963
1-M_AE_nT                                   0.259532
DAILY_SUNSPOT_NO_                           0.631548
DAILY_F10.7_                                0.513907
SOLAR_LYMAN-ALPHA_W/m^2                     0.699166
mg_index (core to wing ratio (unitless))    0.681083
irradiance (W/m^2/nm)                       0.705264
storm                                       0.213384
storm phase                                 0.191931
d_diff                                      0.266869
Name: 400kmDensity, dtype: float64